# Jupyter Notebook para Análisis Estadístico de Métricas de Calidad de Código (AP1 vs AP2)

Este notebook está diseñado para realizar un análisis estadístico pareado de la evolución de las métricas de calidad del código entre dos asignaturas del programa (AP1 y AP2).

## Objetivos:

1. **Cargar datos de métricas de calidad** extraídas de SonarCloud para estudiantes en AP1 y AP2
2. **Realizar análisis estadístico pareado** utilizando pruebas t de Student o Wilcoxon según normalidad
3. **Calcular tamaños de efecto** (Cohen's d) para cuantificar la magnitud de los cambios
4. **Aplicar corrección FDR** (False Discovery Rate) para comparaciones múltiples
5. **Generar visualizaciones** comparativas: boxplots, gráficos de evolución pareada, matriz de correlaciones
6. **Producir reportes** en formato Markdown con interpretación de resultados

## Contenido:
- Pruebas estadísticas pareadas (t de Student o Wilcoxon según normalidad)
- Cálculo de tamaños de efecto (Cohen's d)
- Corrección por comparaciones múltiples (FDR Benjamini-Hochberg)
- Visualizaciones: boxplots, gráficos de evolución pareada, matriz de correlaciones
- Generación de reportes en Markdown

In [ ]:
from __future__ import annotations
import os
from dataclasses import dataclass
from typing import List
import numpy as np
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from glob import glob

# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

## 2. Configuración de Métricas y Parámetros

In [ ]:
# Métricas a analizar
METRICS_BASE = [
    "code_smells","bugs","vulnerabilities","security_hotspots",
    "cognitive_complexity","complexity","ncloc","reliability_rating",
    "security_rating","open_issues",
]

# Clasificación de métricas
LOWER_IS_BETTER = {
    "code_smells","bugs","vulnerabilities","security_hotspots",
    "cognitive_complexity","complexity","reliability_rating",
    "security_rating","open_issues"
}
NEUTRAL = set(METRICS_BASE) - LOWER_IS_BETTER

# Parámetros de entrada
CSV_PATH = "https://raw.githubusercontent.com/TesisEnel/Recopilacion_Datos_CalidadCodigo/refs/heads/main/data/Estudiantes_2023-2024_con_metricas_sonarcloud.csv"
OUTPUT_DIR = "outputs"

print(f"Métricas a analizar: {len(METRICS_BASE)}")
print(f"Métricas donde menor es mejor: {len(LOWER_IS_BETTER)}")
print(f"Métricas neutrales: {NEUTRAL}")

## 3. Definición de Clases y Funciones

In [ ]:
@dataclass
class MetricResult:
    metric: str
    test_used: str
    n_paired: int
    mean_ap1: float
    mean_ap2: float
    delta: float
    pct_change: float | None
    p_value: float
    effect_size_d: float | None
    effect_magnitude: str | None
    direction: str
    improved: str
    normality_p: float | None
    
    def to_dict(self):
        return {
            "metric": self.metric,
            "test_used": self.test_used,
            "n_paired": self.n_paired,
            "mean_ap1": self.mean_ap1,
            "mean_ap2": self.mean_ap2,
            "delta_ap2_minus_ap1": self.delta,
            "pct_change": self.pct_change,
            "p_value": self.p_value,
            "effect_size_d": self.effect_size_d,
            "effect_magnitude": self.effect_magnitude,
            "direction": self.direction,
            "improved": self.improved,
            "normality_p": self.normality_p
        }

In [ ]:
def cohen_d_paired(a: np.ndarray, b: np.ndarray) -> float:
    """Calcula el tamaño de efecto Cohen's d para datos pareados"""
    diff = b - a
    sd = diff.std(ddof=1)
    return 0.0 if sd == 0 else diff.mean() / sd

def classify_effect_size(d: float) -> str:
    """Clasifica el tamaño de efecto según Cohen"""
    ad = abs(d)
    if ad < 0.2:
        return "trivial"
    elif ad < 0.5:
        return "pequeño"
    elif ad < 0.8:
        return "mediano"
    else:
        return "grande"

def infer_direction(metric: str) -> str:
    """Determina la dirección de mejora para una métrica"""
    if metric in LOWER_IS_BETTER:
        return "lower_better"
    return "neutral"

def compute_improvement(metric: str, mean_ap1: float, mean_ap2: float) -> str:
    """Determina si hubo mejora según la dirección de la métrica"""
    d = infer_direction(metric)
    if d == "lower_better" and mean_ap2 < mean_ap1:
        return "Yes"
    elif d == "neutral":
        return "Neutral"
    else:
        return "No"

def safe_pct_change(ap1: float, ap2: float) -> float | None:
    """Calcula cambio porcentual de forma segura"""
    return None if ap1 == 0 else (ap2 - ap1) / ap1 * 100.0

## 4. Carga y Preparación de Datos

In [ ]:
def load_dataset(path: str) -> pd.DataFrame:
    """Carga el dataset y convierte las métricas a formato numérico"""
    df = pd.read_csv(path)
    for m in METRICS_BASE:
        for suf in ("AP1", "AP2"):
            col = f"{m}_{suf}"
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

# Cargar datos
df = load_dataset(CSV_PATH)
print(f"Dataset cargado: {df.shape[0]} estudiantes")
print(f"\nPrimeras columnas: {list(df.columns[:10])}")
df.head()

## 5. Análisis Estadístico por Métrica

In [ ]:
def analyze_metric(df: pd.DataFrame, metric: str) -> MetricResult:
    """Realiza análisis estadístico pareado para una métrica"""
    col_ap1 = f"{metric}_AP1"
    col_ap2 = f"{metric}_AP2"
    
    # Verificar que existan las columnas
    if col_ap1 not in df.columns or col_ap2 not in df.columns:
        return MetricResult(
            metric, "NA", 0, np.nan, np.nan, np.nan, None, np.nan, None, 
            infer_direction(metric), "NA", None, None
        )
    
    # Preparar datos pareados
    data = df[[col_ap1, col_ap2]].dropna()
    a = data[col_ap1].values
    b = data[col_ap2].values
    n = len(data)
    
    # Verificar tamaño mínimo de muestra
    if n < 3:
        return MetricResult(
            metric, "Insuficiente", n, float("nan"), float("nan"), float("nan"),
            None, float("nan"), None, infer_direction(metric), "NA", None, None
        )
    
    # Test de normalidad en las diferencias
    diffs = b - a
    try:
        _, p_norm = stats.shapiro(diffs)
    except Exception:
        p_norm = None
    
    # Seleccionar prueba estadística
    if p_norm is not None and p_norm > 0.05:
        _, p_val = stats.ttest_rel(a, b, nan_policy='omit')
        test_used = "paired_t"
    else:
        if np.allclose(diffs, 0):
            p_val = 1.0
            test_used = "wilcoxon_allzero"
        else:
            try:
                _, p_val = stats.wilcoxon(a, b, zero_method='wilcox', alternative='two-sided')
                test_used = "wilcoxon"
            except ValueError:
                p_val = 1.0
                test_used = "wilcoxon_error"
    
    # Calcular estadísticos
    mean_ap1 = float(np.mean(a))
    mean_ap2 = float(np.mean(b))
    delta = mean_ap2 - mean_ap1
    pct = safe_pct_change(mean_ap1, mean_ap2)
    d = cohen_d_paired(a, b)
    magnitude = classify_effect_size(d)
    improved = compute_improvement(metric, mean_ap1, mean_ap2)
    
    return MetricResult(
        metric, test_used, n, mean_ap1, mean_ap2, delta, pct, p_val, d,
        magnitude, infer_direction(metric), improved, p_norm
    )

In [ ]:
def run_analysis(df: pd.DataFrame) -> pd.DataFrame:
    """Ejecuta el análisis para todas las métricas y aplica corrección FDR"""
    res = [analyze_metric(df, m) for m in METRICS_BASE]
    res_df = pd.DataFrame([r.to_dict() for r in res])
    
    # Aplicar corrección FDR
    mask = res_df["p_value"].notna()
    pvals = res_df.loc[mask, "p_value"].values
    
    if len(pvals) > 0:
        rejected, p_corr, _, _ = multipletests(pvals, alpha=0.05, method='fdr_bh')
        res_df.loc[mask, "p_value_fdr"] = p_corr
        res_df.loc[mask, "significant_raw"] = res_df.loc[mask, "p_value"] < 0.05
        res_df.loc[mask, "significant_fdr"] = rejected
    
    return res_df

# Ejecutar análisis
print("Ejecutando análisis estadístico...")
res_df = run_analysis(df)
print("✓ Análisis completado")

## 6. Resultados del Análisis

In [ ]:
# Ordenar por p-valor corregido
res_sorted = res_df.sort_values("p_value_fdr") if "p_value_fdr" in res_df.columns else res_df.sort_values("p_value")

# Mostrar resultados principales
cols_show = [
    "metric", "n_paired", "mean_ap1", "mean_ap2", "delta_ap2_minus_ap1", 
    "pct_change", "test_used", "p_value", "p_value_fdr", 
    "effect_size_d", "effect_magnitude", "improved"
]

print("\n" + "="*80)
print("RESUMEN DE MÉTRICAS (ordenadas por p-valor corregido)")
print("="*80)
display(res_sorted[cols_show])

In [ ]:
# Resumen estadístico
sig = res_sorted[res_sorted.significant_fdr == True]
improved = sig[sig.improved == 'Yes']
worsened = sig[sig.improved == 'No']
neutral = sig[sig.improved == 'Neutral']

print("\n" + "="*80)
print("RESUMEN EJECUTIVO")
print("="*80)
print(f"Total de métricas analizadas: {len(res_sorted)}")
print(f"Métricas con cambios significativos (FDR ≤ 0.05): {len(sig)}")
print(f"\n  → Mejoras significativas: {len(improved)}")
if len(improved) > 0:
    print(f"     {', '.join(improved.metric.tolist())}")
print(f"\n  → Deterioros significativos: {len(worsened)}")
if len(worsened) > 0:
    print(f"     {', '.join(worsened.metric.tolist())}")
print(f"\n  → Cambios neutrales: {len(neutral)}")
if len(neutral) > 0:
    print(f"     {', '.join(neutral.metric.tolist())}")

## 7. Guardar Resultados

In [ ]:
# Crear directorio de salida
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Guardar CSVs
out_raw = os.path.join(OUTPUT_DIR, "resultados_metricas.csv")
out_fdr = os.path.join(OUTPUT_DIR, "resultados_metricas_fdr.csv")

res_df.to_csv(out_raw, index=False)
res_sorted.to_csv(out_fdr, index=False)

print(f"✓ Resultados guardados:")
print(f"  - {out_raw}")
print(f"  - {out_fdr}")

## 8. Visualizaciones

### 8.1 Boxplots Comparativos

In [ ]:
def plot_boxplots(df: pd.DataFrame, outdir: str, metrics: List[str]):
    """Genera boxplots comparativos para todas las métricas"""
    rows = []
    for m in metrics:
        c1, c2 = f"{m}_AP1", f"{m}_AP2"
        if c1 in df.columns and c2 in df.columns:
            sub = df[[c1, c2]].copy()
            sub.columns = ["AP1", "AP2"]
            if sub.dropna().empty:
                continue
            long = sub.melt(var_name="asignatura", value_name="valor")
            long["metric"] = m
            rows.append(long)
    
    if not rows:
        return
    
    long_df = pd.concat(rows, ignore_index=True)
    metrics_present = sorted(long_df["metric"].unique())
    n_metrics = len(metrics_present)
    ncols = 4
    nrows = int(np.ceil(n_metrics / ncols))
    
    fig, axes = plt.subplots(nrows, ncols, figsize=(4*ncols, 4*nrows))
    axes = np.atleast_2d(axes).reshape(nrows, ncols)
    
    for ax, metric in zip(axes.flat, metrics_present):
        g = long_df[long_df.metric == metric]
        sns.boxplot(data=g, x="asignatura", y="valor", ax=ax)
        sns.stripplot(data=g, x="asignatura", y="valor", ax=ax, 
                     color="#555", alpha=0.4, jitter=0.2, size=3)
        ax.set_title(metric)
    
    # Desactivar ejes sobrantes
    for ax in axes.flat[len(metrics_present):]:
        ax.axis('off')
    
    plt.tight_layout()
    out = os.path.join(outdir, "fig_boxplots.png")
    plt.savefig(out, dpi=150)
    print(f"✓ Boxplots guardados: {out}")
    plt.show()

plot_boxplots(df, OUTPUT_DIR, METRICS_BASE)

### 8.2 Gráficos de Evolución Pareada (Spaghetti Plots)

In [ ]:
def plot_spaghetti(df: pd.DataFrame, outdir: str, metrics: List[str], max_plots: int = 6):
    """Genera gráficos de evolución pareada para las métricas más relevantes"""
    count = 0
    for m in metrics:
        if count >= max_plots:
            break
            
        c1, c2 = f"{m}_AP1", f"{m}_AP2"
        if c1 not in df.columns or c2 not in df.columns:
            continue
        
        sub = df[[c1, c2]].dropna()
        if sub.empty:
            continue
        
        fig, ax = plt.subplots(figsize=(6, 5))
        x = [1, 2]
        
        # Líneas de conexión
        for _, row in sub.iterrows():
            ax.plot(x, [row[c1], row[c2]], color="#999", alpha=0.5, linewidth=0.8)
        
        # Puntos
        ax.scatter([1]*len(sub), sub[c1], color="#1f77b4", label="AP1", s=40, alpha=0.7)
        ax.scatter([2]*len(sub), sub[c2], color="#ff7f0e", label="AP2", s=40, alpha=0.7)
        
        ax.set_xticks(x)
        ax.set_xticklabels(["AP1", "AP2"])
        ax.set_title(f"Evolución pareada: {m}", fontsize=12, fontweight='bold')
        ax.set_ylabel("Valor", fontsize=10)
        ax.grid(alpha=0.3, linestyle='--')
        ax.legend(frameon=False)
        
        plt.tight_layout()
        out = os.path.join(outdir, f"fig_spaghetti_{m}.png")
        plt.savefig(out, dpi=130)
        plt.show()
        count += 1
    
    print(f"\n✓ {count} gráficos de evolución pareada generados")

# Generar para las 6 métricas más relevantes (las que mostraron cambios significativos)
top_metrics = res_sorted.head(6).metric.tolist()
plot_spaghetti(df, OUTPUT_DIR, top_metrics, max_plots=6)

### 8.3 Matriz de Correlaciones

In [ ]:
def plot_correlation_heatmap(df: pd.DataFrame, outdir: str, metrics: List[str]):
    """Genera matriz de correlaciones entre métricas AP1 y AP2"""
    cols = []
    for m in metrics:
        for suf in ("AP1", "AP2"):
            c = f"{m}_{suf}"
            if c in df.columns:
                cols.append(c)
    
    if not cols:
        return
    
    corr_df = df[cols].copy()
    if corr_df.empty:
        return
    
    corr = corr_df.corr()
    
    fig_size = min(1 + 0.5 * len(corr.columns), 18)
    plt.figure(figsize=(fig_size, fig_size))
    
    sns.heatmap(corr, cmap="coolwarm", center=0, annot=False, 
                linewidths=0.3, cbar_kws={'label': 'Correlación'})
    plt.title("Matriz de Correlaciones (AP1 & AP2)", fontsize=14, fontweight='bold')
    plt.tight_layout()
    
    out = os.path.join(outdir, "fig_heatmap_correlaciones.png")
    plt.savefig(out, dpi=160)
    print(f"✓ Matriz de correlaciones guardada: {out}")
    plt.show()

plot_correlation_heatmap(df, OUTPUT_DIR, METRICS_BASE)

## 9. Generación de Reportes en Markdown

In [ ]:
def format_pct(x):
    """Formatea un valor como porcentaje"""
    try:
        if x is None or (isinstance(x, float) and pd.isna(x)):
            return "NA"
        return f"{float(x):.1f}%"
    except Exception:
        return "NA"

def generate_markdown_report(res_df: pd.DataFrame, outdir: str, metrics: List[str], csv_path: str):
    """Genera reporte detallado en Markdown"""
    ts = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
    out_path = os.path.join(outdir, 'reporte_metricas.md')
    
    df = res_df[res_df.metric.isin(metrics)].copy()
    sig = df[df.significant_fdr == True]
    improved_sig = sig[sig.improved == 'Yes']
    worsened_sig = sig[sig.improved == 'No']
    
    df['abs_d'] = df['effect_size_d'].abs()
    top_effect = df.sort_values('abs_d', ascending=False).head(5)
    
    lines = []
    lines.append(f"# Reporte Estadístico de Métricas AP1 vs AP2\n")
    lines.append(f"Generado: {ts}\n")
    lines.append(f"Fuente CSV: `{csv_path}`\n")
    lines.append("## Resumen Global\n")
    
    total = len(df)
    sig_n = sig.shape[0]
    lines.append(f"Se analizaron {total} métricas. {sig_n} resultaron significativas tras corrección FDR (α=0.05).\n")
    
    if improved_sig.shape[0] > 0:
        lines.append(f"- Mejoras significativas: {improved_sig.shape[0]} -> {', '.join(improved_sig.metric)}")
    if worsened_sig.shape[0] > 0:
        lines.append(f"- Deterioros significativos: {worsened_sig.shape[0]} -> {', '.join(worsened_sig.metric)}")
    
    neutrals = sig[sig.improved == 'Neutral']
    if neutrals.shape[0] > 0:
        lines.append(f"- Cambios significativos pero neutros (contexto): {neutrals.shape[0]} -> {', '.join(neutrals.metric)}")
    
    lines.append("\n## Principales Cambios (Top |d|)\n")
    for _, r in top_effect.iterrows():
        direction = '↓' if r.direction == 'lower_better' and r.mean_ap2 < r.mean_ap1 else ('↑' if r.direction == 'higher_better' and r.mean_ap2 > r.mean_ap1 else '↔')
        lines.append(f"- {r.metric}: d={r.effect_size_d:.3f} ({r.effect_magnitude}), p_FDR={r.p_value_fdr if not pd.isna(r.p_value_fdr) else r.p_value:.3g}, {direction} cambio relativo {format_pct(r.pct_change)} (AP1={r.mean_ap1:.3g}, AP2={r.mean_ap2:.3g}) -> Improved={r.improved}")
    
    lines.append("\n## Tabla Detallada\n")
    show_cols = ["metric", "mean_ap1", "mean_ap2", "pct_change", "test_used", "p_value", "p_value_fdr", "effect_size_d", "effect_magnitude", "improved"]
    header = '|' + '|'.join(show_cols) + '|'
    sep = '|' + '|'.join(['---'] * len(show_cols)) + '|'
    lines.append(header)
    lines.append(sep)
    
    for _, r in df.sort_values('p_value_fdr').iterrows():
        def safe(v, fmt=None):
            if v is None or (isinstance(v, float) and pd.isna(v)):
                return 'NA'
            try:
                return fmt.format(v) if fmt else str(v)
            except Exception:
                return str(v)
        
        lines.append('|' + '|'.join([
            r.metric,
            safe(r.mean_ap1, "{:.3g}"),
            safe(r.mean_ap2, "{:.3g}"),
            format_pct(r.pct_change),
            r.test_used if isinstance(r.test_used, str) else 'NA',
            safe(r.p_value, "{:.3g}"),
            safe(r.p_value_fdr, "{:.3g}"),
            safe(r.effect_size_d, "{:.3g}"),
            r.effect_magnitude if isinstance(r.effect_magnitude, str) else 'NA',
            r.improved if isinstance(r.improved, str) else 'NA'
        ]) + '|')
    
    lines.append("\n## Interpretación General\n")
    if improved_sig.shape[0] > 0:
        lines.append(f"Las métricas con mejoras significativas muestran evidencia de impacto positivo (ej. {', '.join(improved_sig.metric[:3])}{'...' if improved_sig.shape[0] > 3 else ''}).")
    if worsened_sig.shape[0] > 0:
        lines.append(f"Atención: algunas métricas empeoraron significativamente (ej. {', '.join(worsened_sig.metric[:3])}).")
    lines.append("Los tamaños de efecto clasificados como medianos indican cambios sustanciales prácticos; revisar contexto pedagógico.")
    
    with open(out_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(lines))
    
    return out_path

# Generar reporte
md_path = generate_markdown_report(res_sorted, OUTPUT_DIR, METRICS_BASE, CSV_PATH)
print(f"✓ Reporte Markdown generado: {md_path}")

In [ ]:
def generate_executive_summary(res_df: pd.DataFrame, outdir: str):
    """Genera resumen ejecutivo"""
    path = os.path.join(outdir, 'resumen_ejecutivo.md')
    sig = res_df[res_df.significant_fdr == True]
    improvements = ', '.join(sig[sig.improved == 'Yes'].metric.tolist()[:4])
    deterioro = ', '.join(sig[sig.improved == 'No'].metric.tolist()[:4])
    
    lines = [
        "# Resumen Ejecutivo",
        "",
        "## Claves",
        f"Cambios significativos tras FDR: {len(sig)}",
        f"Mejoras: {improvements if improvements else 'Ninguna'}",
        f"Deterioros: {deterioro if deterioro else 'Ninguno'}",
        "",
        "## Visuales"
    ]
    
    for img in ["fig_boxplots.png", "fig_heatmap_correlaciones.png"]:
        if os.path.exists(os.path.join(outdir, img)):
            lines.append(f"![{img}]({img})")
    
    with open(path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(lines))
    
    return path

# Generar resumen ejecutivo
exec_path = generate_executive_summary(res_sorted, OUTPUT_DIR)
print(f"✓ Resumen ejecutivo generado: {exec_path}")

## 10. Conclusiones

El análisis ha completado exitosamente. Los archivos generados incluyen:

- **CSVs de resultados**: `resultados_metricas.csv` y `resultados_metricas_fdr.csv`
- **Visualizaciones**: Boxplots, gráficos de evolución pareada y matriz de correlaciones
- **Reportes**: Reporte detallado y resumen ejecutivo en Markdown

Revisa los resultados en el directorio `outputs/`.